# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())


# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify \
                    WITH REPLICATION = \
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Table Name: songs_by_session
**Purpose:**
- To retrieve song play details (artist name, song name, song length) for a specific session and item within that session.

**Primary Key:**
- **Partition Key:**
`session_id`
-> This partitions the data by session, meaning all data for the same session goes to the same node.

- **Clustering Key:**
`item_in_session`
-> This orders songs within a session by the sequence in which they were played, enabling queries like "which song was item 4 in session 338"

**Reasoning:**
- This schema supports fast, efficient lookups using a known `session_id` and `item_in_session`(as per query requirement)
- It enables us to uniquely identify each song play in a session by its order(ItemInSession)

In [8]:
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS songs_by_session"
query = query + "(session_id int, item_in_session int, artist_name text, \
                    song_name text, song_length float, \
                    PRIMARY KEY (session_id, item_in_session));"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session (session_id, item_in_session,artist_name, song_name, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist_name, song_name, song_length FROM songs_by_session WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Faithless', song_name='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### Table Name: user_session_playlist
**Purpose:**
- To retrieve the playlist of a specific user during a specific session including the artist name, song title(ordered by play sequence), and user's first and last name

**Primary Key:**
- **Partition Key:**
`(user_id, session_id)`
-> Partitions the data by user and session, grouping all song plays from a particular user during a specific session together.

- **Clustering Key:**
`item_in_session`
-> Orders the songs within the session by the order in which they were played.

**Reasoning:**
- This schema allows efficient queries when both `user_id` and `session_id` are known.
- It enables retrieving an ordered playlist by using `item_in_Session` as the clustering key

In [11]:
## for userid = 10, sessionid = 182

query1 = "CREATE TABLE IF NOT EXISTS user_session_playlist"
query1 = query1 + "(\
                    user_id int, \
                    session_id int, \
                    item_in_session int,\
                    user_fname text, \
                    user_lname text, \
                    artist_name text, \
                    song_name text,\
                    PRIMARY KEY ((user_id, session_id), item_in_session));"

try:
    session.execute(query1)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_playlist (user_id, session_id, item_in_session, user_fname, user_lname, artist_name, song_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))



In [12]:
query = "SELECT artist_name, song_name, user_fname, user_lname FROM user_session_playlist WHERE user_id = 10 and session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_name="Keep On Keepin' On", user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Three Drives', song_name='Greece 2000', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Sebastien Tellier', song_name='Kilometer', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Lonnie Gordon', song_name='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_fname='Sylvie', user_lname='Cruz')


### Table Name: song_listeners
**Purpose:**
- To retrieve the full names (first and last) of all users who listened to a specific song.

**Primary Key:**
- **Partition Key:**
`song_name`
-> Groups all user records by song, allowing efficient lookup of listeners for a given song title.

- **Clustering Key:**
`user_id`
-> Ensures uniqueness of each user per song and allows future extensibility (e.g., to track listen count or timestamp)

**Reasoning:**
- This schema is optimized for queries that need to find all users who listened to a specific song.
- Using `song_name` as the partition key enables quick access to all listeners of a given song, while `user_id` prevents duplicates and supports potential expansion.

In [13]:
query3 = "CREATE TABLE IF NOT EXISTS song_listeners"
query3 = query3 + "(song_name text, \
                    user_id int, \
                    user_fname text, \
                    user_lname text, \
                    PRIMARY KEY (song_name, user_id));"

try:
    session.execute(query3)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song_name, user_id, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))




In [14]:
query = "SELECT user_fname, user_lname FROM song_listeners WHERE song_name = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user_fname='Jacqueline', user_lname='Lynch')
Row(user_fname='Tegan', user_lname='Levine')
Row(user_fname='Sara', user_lname='Johnson')


### Drop the tables before closing out the sessions

In [15]:
query1 = ("DROP TABLE IF EXISTS songs_by_session")
query2 = ("DROP TABLE IF EXISTS user_session_playlist")
query3 = ("DROP TABLE IF EXISTS song_listeners")

try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()